In [1]:
import numpy as np
import pandas as pd
import yfinance as yf

1、获取股票列表

In [2]:
df = pd.read_excel('./股票列表.xlsx', engine='openpyxl')
code_list = []
# 遍历第二列中的股票代码
for stock_code in df['代码']:
    old_code = stock_code
    if stock_code.startswith('SH'):
        stock_code = f"{stock_code[2:]}.{'SS'}"
    elif stock_code.startswith('SZ'):
        stock_code = f"{stock_code[2:]}.{'SZ'}"
    code_list.append(stock_code)
code_list

['300905.SZ',
 '300561.SZ',
 '300287.SZ',
 '300125.SZ',
 '300086.SZ',
 '300386.SZ',
 '000566.SZ',
 '000850.SZ',
 '002036.SZ',
 '603123.SS',
 '600676.SS',
 '600841.SS',
 '601162.SS',
 '600148.SS',
 '600650.SS',
 '600635.SS',
 '605218.SS',
 '600847.SS',
 '000062.SZ',
 '603883.SS',
 '002104.SZ',
 '001298.SZ',
 '000750.SZ',
 '000678.SZ',
 '000627.SZ',
 '603306.SS',
 '600838.SS',
 '600611.SS',
 '603626.SS',
 '002823.SZ',
 '600826.SS',
 '002640.SZ',
 '002520.SZ',
 '600828.SS',
 '002717.SZ',
 '300736.SZ',
 '301607.SZ',
 '002456.SZ',
 '300641.SZ',
 '600418.SS',
 '300209.SZ',
 '300758.SZ',
 '605289.SS',
 '001696.SZ',
 '300077.SZ',
 '300464.SZ',
 '600686.SS',
 '002388.SZ',
 '601890.SS',
 '601086.SS',
 '300013.SZ',
 '605598.SS',
 '002827.SZ',
 '600769.SS',
 '002208.SZ',
 '600576.SS',
 '300518.SZ',
 '300619.SZ',
 '300436.SZ',
 '603036.SS',
 '300068.SZ',
 '002312.SZ',
 '002528.SZ',
 '600898.SS',
 '000536.SZ',
 '000712.SZ',
 '600358.SS',
 '000908.SZ',
 '000584.SZ',
 '603559.SS',
 '605228.SS',
 '6008

2、定义起始日期

In [3]:
start_date = '2015-01-01'
end_date = '2024-08-31'

3、获取所有股票的数据整合到一个文件中

In [4]:
df = pd.read_csv('./all_data.csv')
df

,Unnamed: 0,Date,Open,High,Low,Close,Adj Close,Volume,code
0,0,2020-11-05,55.000000,62.150002,51.000000,51.029999,47.681602,21472552.0,300905.SZ
1,1,2020-11-06,47.005001,51.520000,46.505001,48.599998,45.411049,13282442.0,300905.SZ
2,2,2020-11-09,48.500000,49.845001,47.119999,48.740002,45.541870,11263252.0,300905.SZ
3,3,2020-11-10,48.000000,49.439999,47.000000,47.250000,44.149632,8399338.0,300905.SZ
4,4,2020-11-11,46.900002,49.055000,46.595001,47.700001,44.570107,8407878.0,300905.SZ
...,...,...,...,...,...,...,...,...,...
195256,195256,2024-08-26,10.470000,11.520000,10.420000,11.520000,11.520000,10574100.0,600506.SS
195257,195257,2024-08-27,11.500000,11.770000,11.220000,11.280000,11.280000,13423101.0,600506.SS
195258,195258,2024-08-28,11.380000,12.300000,11.210000,11.600000,11.600000,13100100.0,600506.SS
195259,195259,2024-08-29,11.530000,11.680000,11.160000,11.350000,11.350000,9205800.0,600506.SS


4、数据清洗

In [5]:
# 过滤掉code列中以'300'开头的行
df = df[~df['code'].str.startswith('300')]
df = df[~df['code'].str.startswith('688')]
df = df[~df['code'].str.startswith('8')]

df
df = df.iloc[:5000]

In [6]:
df.set_index('code', inplace=True)

In [7]:
# 计算每只股票的当日涨幅
df['Pct_Chg'] = df.groupby(level=0)['Close'].pct_change()
# 计算3天后涨幅
df['Pct_Chg_3d_later'] = df.groupby(level=0)['Close'].apply(lambda x: (x.shift(-3) /x ) - 1) * 100
# 计算5天后涨幅
df['Pct_Chg_5d_later'] = df.groupby(level=0)['Close'].apply(lambda x: (x.shift(-5) /x ) - 1) * 100
# 计算10天后涨幅
df['Pct_Chg_10d_later'] = df.groupby(level=0)['Close'].apply(lambda x: (x.shift(-10) /x ) - 1) * 100
# 计算20天后涨幅
df['Pct_Chg_20d_later'] = df.groupby(level=0)['Close'].apply(lambda x: (x.shift(-20) /x ) - 1) * 100
# 将涨幅数据转换为百分比形式
df['Pct_Chg'] *= 100
# 去除Pct_Chg为空的值
df = df.dropna(subset=['Pct_Chg','Pct_Chg_3d_later', 'Pct_Chg_5d_later', 'Pct_Chg_10d_later', 'Pct_Chg_20d_later'])
df

,Unnamed: 0,Date,Open,High,Low,Close,Adj Close,Volume,Pct_Chg,Pct_Chg_3d_later,Pct_Chg_5d_later,Pct_Chg_10d_later,Pct_Chg_20d_later
code,,,,,,,,,,,,,
000566.SZ,11835,2015-01-06,7.390000,7.710000,7.365000,7.595000,7.396634,14522622.0,2.843596,4.608300,3.620805,12.113240,25.082294
000566.SZ,11836,2015-01-07,7.645000,8.050000,7.645000,7.755000,7.552455,24060822.0,2.106653,3.997414,1.353965,15.473885,23.146358
000566.SZ,11837,2015-01-08,7.800000,7.885000,7.605000,7.705000,7.503761,10945550.0,-0.644748,2.141466,1.557429,15.963667,22.777419
000566.SZ,11838,2015-01-09,7.845000,8.090000,7.845000,7.945000,7.737493,25795612.0,3.114864,-1.069856,-1.699185,10.195090,14.222777
000566.SZ,11839,2015-01-12,7.945000,8.100000,7.730000,8.065000,7.854358,17574292.0,1.510376,-2.975819,-2.665836,13.329210,14.631127
...,...,...,...,...,...,...,...,...,...,...,...,...,...
002036.SZ,16809,2018-02-07,7.810650,7.982248,7.532544,7.869822,7.781206,15089185.0,5.304835,6.616535,4.812026,26.616543,27.744364
002036.SZ,16810,2018-02-08,7.816568,8.402366,7.769230,8.195266,8.102985,13687642.0,4.135338,0.433218,1.227436,22.743679,20.866427
002036.SZ,16811,2018-02-09,7.934911,8.372781,7.899408,8.307692,8.214146,12627806.0,1.371838,-0.712252,-1.638181,17.307696,19.088326


5、和上证指数数据合并

In [8]:
df_szzs = pd.read_csv('000001.csv')
df_szzs.set_index('code', inplace=True)
# 计算每只股票的当日涨幅
df_szzs['Pct_Chg_szzs'] = df_szzs.groupby(level=0)['Close'].pct_change()
# 将涨幅数据转换为百分比形式
df_szzs['Pct_Chg_szzs'] *= 100
# 去除Pct_Chg为空的值
df_szzs = df_szzs.dropna(subset=['Pct_Chg_szzs'])

df_szzs

,Date,Open,High,Low,Close,Adj Close,Volume,Pct_Chg_szzs
code,,,,,,,,
000001.SS,2015-01-06,3330.799072,3394.224121,3303.184082,3351.446045,3351.446045,501700,0.027667
000001.SS,2015-01-07,3326.648926,3374.895996,3312.210938,3373.954102,3373.954102,391900,0.671592
000001.SS,2015-01-08,3371.957031,3381.565918,3285.094971,3293.456055,3293.456055,371100,-2.385867
000001.SS,2015-01-09,3276.965088,3404.833984,3267.509033,3285.412109,3285.412109,410200,-0.244240
000001.SS,2015-01-12,3258.212891,3275.185059,3191.582031,3229.315918,3229.315918,322100,-1.707432
...,...,...,...,...,...,...,...,...
000001.SS,2024-08-26,2855.474121,2860.655029,2844.383057,2855.520020,2855.520020,227400,0.040363
000001.SS,2024-08-27,2850.876953,2853.780029,2843.157959,2848.730957,2848.730957,230700,-0.237752
000001.SS,2024-08-28,2844.326904,2852.562012,2830.961914,2837.428955,2837.428955,228700,-0.396738


In [9]:
# 重置索引，以便合并
df = df.reset_index()
df_szzs = df_szzs.reset_index()
# 合并两个DataFrame
# 这里我们按照'Date'列进行合并，并且使用左连接，这样只会保留df中的日期
df = df.merge(df_szzs[['Date', 'Pct_Chg_szzs']], on='Date', how='left')
df

,code,Unnamed: 0,Date,Open,High,Low,Close,Adj Close,Volume,Pct_Chg,Pct_Chg_3d_later,Pct_Chg_5d_later,Pct_Chg_10d_later,Pct_Chg_20d_later,Pct_Chg_szzs
0,000566.SZ,11835,2015-01-06,7.390000,7.710000,7.365000,7.595000,7.396634,14522622.0,2.843596,4.608300,3.620805,12.113240,25.082294,0.027667
1,000566.SZ,11836,2015-01-07,7.645000,8.050000,7.645000,7.755000,7.552455,24060822.0,2.106653,3.997414,1.353965,15.473885,23.146358,0.671592
2,000566.SZ,11837,2015-01-08,7.800000,7.885000,7.605000,7.705000,7.503761,10945550.0,-0.644748,2.141466,1.557429,15.963667,22.777419,-2.385867
3,000566.SZ,11838,2015-01-09,7.845000,8.090000,7.845000,7.945000,7.737493,25795612.0,3.114864,-1.069856,-1.699185,10.195090,14.222777,-0.244240
4,000566.SZ,11839,2015-01-12,7.945000,8.100000,7.730000,8.065000,7.854358,17574292.0,1.510376,-2.975819,-2.665836,13.329210,14.631127,-1.707432
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4932,002036.SZ,16809,2018-02-07,7.810650,7.982248,7.532544,7.869822,7.781206,15089185.0,5.304835,6.616535,4.812026,26.616543,27.744364,-1.821371
4933,002036.SZ,16810,2018-02-08,7.816568,8.402366,7.769230,8.195266,8.102985,13687642.0,4.135338,0.433218,1.227436,22.743679,20.866427,-1.426602
4934,002036.SZ,16811,2018-02-09,7.934911,8.372781,7.899408,8.307692,8.214146,12627806.0,1.371838,-0.712252,-1.638181,17.307696,19.088326,-4.052635
4935,002036.SZ,16812,2018-02-12,8.325443,8.491124,8.183431,8.390532,8.296052,9932084.0,0.997148,-1.128345,7.122712,16.925249,17.559953,0.775562


In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

线性回归预测当日涨幅和大盘涨幅对3日收益率的影响          
探究大盘下跌时股价逆势上涨是否更容易导致后续的股价增长

In [11]:
def linear_model_1_Normal_Equation(df,x_target):
     """
     线性回归：正规方程
     """
     # 1. 选择特征变量和目标变量
     target = x_target  # 目标变量
     features = ['Pct_Chg','Pct_Chg_szzs']  # 特征变量

     # 2. 数据集划分
     x_train, x_test, y_train, y_test = train_test_split(df[features], df[target]-df['Pct_Chg'], random_state=2, test_size=0.2)

     # 3. 特征工程：标准化
     transformer = StandardScaler()
     
     # fit()函数用于计算训练数据的均值和标准差，以便对训练数据进行标准化
     x_train = transformer.fit_transform(x_train)

     # transform()函数用于使用相同的均值和标准差来对测试数据进行标准化。这确保了我们使用相同的标准化方法来处理训练和测试数据。
     x_test = transformer.transform(x_test)
     
     # 4. 机器学习：线性回归（正规方程）
     estimator = LinearRegression()
     estimator.fit(x_train, y_train)
     
     # 5. 模型评估
     ## 5.1 获取系数
     y_predict = estimator.predict(x_test)
     print(f"预测值为：{y_predict}")
     print(f"模型中的系数为：{estimator.coef_}")
     print(f"模型中的偏置为：{estimator.intercept_}")
     
     ## 5.2 评价：均方误差
     error_mse = mean_squared_error(y_test, y_predict)
     print(f"均方误差为：{error_mse}")
     
     return None


In [12]:
x_target = 'Pct_Chg_3d_later'
linear_model_1_Normal_Equation(df,x_target)

预测值为：[ 5.61293300e-01  2.07829428e-01  2.27965354e+00  2.41264462e+00
  4.15348634e+00  1.82012372e+00 -2.24401402e-01  1.68765147e+00
  1.58687497e+00 -1.20880489e+00  1.16700180e+00 -1.94545013e+00
  4.74273279e+00  1.77737243e+00  1.50329695e+00 -4.59099043e-02
 -1.89525891e+00 -1.24349314e+00  3.62067802e+00 -1.76969106e+00
  1.43368436e+00  2.72811575e-01 -2.58454261e+00  7.44381408e-01
  1.58049707e-01  1.00884055e+00 -6.76853679e-01 -6.31866245e-02
  1.46438393e+00 -2.86267404e+00  7.13453140e-01  5.21621553e-01
 -9.99551122e-01  3.74846339e+00  7.19252358e-02 -5.30771446e-02
  1.74725670e+00 -1.16456663e+00 -1.08825526e+00  3.22683154e+00
 -8.25903453e+00 -1.16027711e-01 -3.11299895e+00 -2.88997397e-01
  3.95003735e-01  2.27302409e+00 -1.63856652e+00  1.04559049e+00
 -3.22635745e+00  1.11431547e+00  6.90528560e-01  1.77954347e+00
  3.46144834e+00  4.11900050e-01  4.07496843e+00  2.19768265e-01
 -2.34018621e+00  9.74329827e-03 -4.07420027e+00  1.76619942e+00
  2.24644684e+00 -2.

探究大盘下跌情况下逆势涨停的股票后续是否可能会更大幅度上涨

In [13]:
df_best = df[df['Pct_Chg'] > 9.8]
x_target = 'Pct_Chg_3d_later'
print(f'************逆势上涨直至涨停的股票***************')
linear_model_1_Normal_Equation(df_best,x_target)
df_lost = df[df['Pct_Chg'] < 9.8]
print(f'************逆势上涨但未涨停的股票***************')
linear_model_1_Normal_Equation(df_lost,x_target)

************逆势上涨直至涨停的股票***************
预测值为：[-4.51084984 -4.16416251 -4.79445684 -4.7986034  -4.54320097 -4.74816929
 -4.36403509 -4.43511187 -4.58725324 -4.42649037 -4.84291798 -5.16056838
 -4.54290354 -4.7525867  -4.52704401 -4.64927656]
模型中的系数为：[-2.65277702  0.21125589]
模型中的偏置为：-5.04560878397043
均方误差为：202.21699377257602
************逆势上涨但未涨停的股票***************
预测值为：[ 1.21466497e+00  3.62518189e+00 -6.35622245e-01  1.58634229e+00
 -9.36000677e-01 -1.09343760e+00 -1.84145052e+00 -3.32709046e+00
  9.73008222e-01 -1.08890643e+00  1.55358228e+00  2.15373467e+00
 -1.23165122e-01  2.15399142e-01 -6.38453733e+00  1.29083541e+00
  3.85073789e+00  1.06335137e+00  1.40856415e+00 -1.79126614e+00
 -9.45307120e-01  5.60329813e-01  9.60868510e-01  3.20830071e+00
 -1.05741499e+00  9.55794326e-01  1.79543821e+00 -1.24102910e+00
  2.57101755e+00  3.72176457e+00 -5.36661920e+00  7.97101511e-02
 -1.17196378e+00 -2.93636180e+00  1.36897762e+00 -2.39700789e+00
 -1.70522289e+00  4.37501552e-01  4.41561445e+

采用逻辑回归算法进行预测

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [15]:
def logistic_regression_model(data, feature_cols, target_col):
    """
    训练逻辑回归模型并评估性能

    参数:
    data (DataFrame): 包含特征和目标变量的数据集
    feature_cols (list): 特征列名称的列表
    target_col (str): 目标列名称

    返回:
    dict: 包含模型评估指标的字典
    """
    # 提取特征和目标变量
    X = data[feature_cols]
    y = data[target_col]

    # 分割数据集为训练集和测试集
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # 创建逻辑回归模型
    log_regressor = LogisticRegression()
    log_regressor.fit(X_train, y_train)

    # 进行预测
    y_pred = log_regressor.predict(X_test)

    # 计算准确率
    accuracy = accuracy_score(y_test, y_pred)
    
    # 混淆矩阵
    conf_matrix = confusion_matrix(y_test, y_pred)

    # 分类报告
    class_report = classification_report(y_test, y_pred)

    # 返回结果
    return {
        'accuracy': accuracy,
        'confusion_matrix': conf_matrix,
        'classification_report': class_report
    }

探究大盘和当日股价变化对后续股价的预测准确率

In [17]:
print("***********3日*************")
df['Target_3d'] = (df['Pct_Chg_3d_later']> 0).astype(int)
result = logistic_regression_model(df, ['Pct_Chg', 'Pct_Chg_szzs'], 'Target_3d')
print(f"模型准确率: {result['accuracy']}")
print("混淆矩阵:")
print(result['confusion_matrix'])
print("分类报告:")
print(result['classification_report'])
print("***********5日*************")
df['Target_5d'] = (df['Pct_Chg_5d_later']> 0).astype(int)
result = logistic_regression_model(df, ['Pct_Chg', 'Pct_Chg_szzs'], 'Target_5d')
print(f"模型准确率: {result['accuracy']}")
print("混淆矩阵:")
print(result['confusion_matrix'])
print("分类报告:")
print(result['classification_report'])
print("***********10日*************")
df['Target_10d'] = (df['Pct_Chg_10d_later']> 0).astype(int)
result = logistic_regression_model(df, ['Pct_Chg', 'Pct_Chg_szzs'], 'Target_10d')
print(f"模型准确率: {result['accuracy']}")
print("混淆矩阵:")
print(result['confusion_matrix'])
print("分类报告:")
print(result['classification_report'])
print("**********20日*************")
df['Target_20d'] = (df['Pct_Chg_20d_later']> 0).astype(int)
result = logistic_regression_model(df, ['Pct_Chg', 'Pct_Chg_szzs'], 'Target_20d')
print(f"模型准确率: {result['accuracy']}")
print("混淆矩阵:")
print(result['confusion_matrix'])
print("分类报告:")
print(result['classification_report'])
print("**********20内的最高价日*************")
columns_to_check = ['Pct_Chg_3d_later', 'Pct_Chg_5d_later', 'Pct_Chg_10d_later', 'Pct_Chg_20d_later']
df['Max_Pct_Chg'] = df[columns_to_check].max(axis=1)
df['Target_max'] = (df['Max_Pct_Chg']> 0).astype(int)
result = logistic_regression_model(df, ['Pct_Chg', 'Pct_Chg_szzs'], 'Target_max')
print(f"模型准确率: {result['accuracy']}")
print("混淆矩阵:")
print(result['confusion_matrix'])
print("分类报告:")
print(result['classification_report'])

***********3日*************
模型准确率: 0.520242914979757
混淆矩阵:
[[496  10]
 [464  18]]
分类报告:
              precision    recall  f1-score   support

           0       0.52      0.98      0.68       506
           1       0.64      0.04      0.07       482

    accuracy                           0.52       988
   macro avg       0.58      0.51      0.37       988
weighted avg       0.58      0.52      0.38       988

***********5日*************
模型准确率: 0.5091093117408907
混淆矩阵:
[[493   6]
 [479  10]]
分类报告:
              precision    recall  f1-score   support

           0       0.51      0.99      0.67       499
           1       0.62      0.02      0.04       489

    accuracy                           0.51       988
   macro avg       0.57      0.50      0.35       988
weighted avg       0.57      0.51      0.36       988

***********10日*************
模型准确率: 0.4949392712550607
混淆矩阵:
[[483   7]
 [492   6]]
分类报告:
              precision    recall  f1-score   support

           0       0.50    

D:\anaconda3\envs\strategy\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda3\envs\strategy\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda3\envs\strategy\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


1、所有时间段的报告显示负类样本远多于正类样本，导致模型偏向于预测负类。
2、20日最高价虽然准确率很高，但主要是因为模型几乎所有样本都预测为正类。负类样本几乎全被错判，召回率为0。

探究当日股价涨停和大盘对后续股价的预测准确率

In [18]:
df_best = df[df['Pct_Chg'] > 9.8]
print("***********3日*************")
df_best['Target_3d'] = (df_best['Pct_Chg_3d_later'] > 0).astype(int)
result = logistic_regression_model(df_best, ['Pct_Chg', 'Pct_Chg_szzs'], 'Target_3d')
print(f"模型准确率: {result['accuracy']}")
print("混淆矩阵:")
print(result['confusion_matrix'])
print("分类报告:")
print(result['classification_report'])
print("***********5日*************")
df_best['Target_5d'] = (df_best['Pct_Chg_5d_later'] > 0).astype(int)
result = logistic_regression_model(df_best, ['Pct_Chg', 'Pct_Chg_szzs'], 'Target_5d')
print(f"模型准确率: {result['accuracy']}")
print("混淆矩阵:")
print(result['confusion_matrix'])
print("分类报告:")
print(result['classification_report'])
print("***********10日*************")
df_best['Target_10d'] = (df_best['Pct_Chg_10d_later'] > 0).astype(int)
result = logistic_regression_model(df_best, ['Pct_Chg', 'Pct_Chg_szzs'], 'Target_10d')
print(f"模型准确率: {result['accuracy']}")
print("混淆矩阵:")
print(result['confusion_matrix'])
print("分类报告:")
print(result['classification_report'])
print("**********20日*************")
df_best['Target_20d'] = (df_best['Pct_Chg_20d_later'] > 0).astype(int)
result = logistic_regression_model(df_best, ['Pct_Chg', 'Pct_Chg_szzs'], 'Target_20d')
print(f"模型准确率: {result['accuracy']}")
print("混淆矩阵:")
print(result['confusion_matrix'])
print("分类报告:")
print(result['classification_report'])
print("**********20内的最高价日*************")
columns_to_check = ['Pct_Chg_3d_later', 'Pct_Chg_5d_later', 'Pct_Chg_10d_later', 'Pct_Chg_20d_later']
df_best['Max_Pct_Chg'] = df_best[columns_to_check].max(axis=1)
df_best['Target_max'] = (df_best['Max_Pct_Chg'] > 0).astype(int)
result = logistic_regression_model(df_best, ['Pct_Chg', 'Pct_Chg_szzs'], 'Target_max')
print(f"模型准确率: {result['accuracy']}")
print("混淆矩阵:")
print(result['confusion_matrix'])
print("分类报告:")
print(result['classification_report'])

***********3日*************
模型准确率: 0.6875
混淆矩阵:
[[3 5]
 [0 8]]
分类报告:
              precision    recall  f1-score   support

           0       1.00      0.38      0.55         8
           1       0.62      1.00      0.76         8

    accuracy                           0.69        16
   macro avg       0.81      0.69      0.65        16
weighted avg       0.81      0.69      0.65        16

***********5日*************
模型准确率: 0.5625
混淆矩阵:
[[2 6]
 [1 7]]
分类报告:
              precision    recall  f1-score   support

           0       0.67      0.25      0.36         8
           1       0.54      0.88      0.67         8

    accuracy                           0.56        16
   macro avg       0.60      0.56      0.52        16
weighted avg       0.60      0.56      0.52        16

***********10日*************
模型准确率: 0.4375
混淆矩阵:
[[4 6]
 [3 3]]
分类报告:
              precision    recall  f1-score   support

           0       0.57      0.40      0.47        10
           1       0.33      0.5

D:\anaconda3\envs\strategy\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
D:\anaconda3\envs\strategy\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
D:\anaconda3\envs\strategy\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

类别不平衡: 每个时间段的结果显示负类样本的识别能力差，导致模型整体性能受限。负类的召回率普遍很低，而正类的召回率相对较高。

模型性能: 尽管在某些时间段准确率较高，但主要是因为模型偏向于预测正类样本，导致对负类的识别能力不足，可能会导致误导性的高准确率。

探究当日股价略涨和大盘对后续股价的预测准确率

In [19]:
df_lost = df[df['Pct_Chg'] < 9.8]
print("***********3日*************")
df_lost['Target_3d'] = (df_lost['Pct_Chg_3d_later'] > 0).astype(int)
result = logistic_regression_model(df_lost, ['Pct_Chg', 'Pct_Chg_szzs'], 'Target_3d')
print(f"模型准确率: {result['accuracy']}")
print("混淆矩阵:")
print(result['confusion_matrix'])
print("分类报告:")
print(result['classification_report'])
print("***********5日*************")
df_lost['Target_5d'] = (df_lost['Pct_Chg_5d_later'] > 0).astype(int)
result = logistic_regression_model(df_lost, ['Pct_Chg', 'Pct_Chg_szzs'], 'Target_5d')
print(f"模型准确率: {result['accuracy']}")
print("混淆矩阵:")
print(result['confusion_matrix'])
print("分类报告:")
print(result['classification_report'])
print("***********10日*************")
df_lost['Target_10d'] = (df_lost['Pct_Chg_10d_later'] > 0).astype(int)
result = logistic_regression_model(df_lost, ['Pct_Chg', 'Pct_Chg_szzs'], 'Target_10d')
print(f"模型准确率: {result['accuracy']}")
print("混淆矩阵:")
print(result['confusion_matrix'])
print("分类报告:")
print(result['classification_report'])
print("**********20日*************")
df_lost['Target_20d'] = (df_lost['Pct_Chg_20d_later'] > 0).astype(int)
result = logistic_regression_model(df_lost, ['Pct_Chg', 'Pct_Chg_szzs'], 'Target_20d')
print(f"模型准确率: {result['accuracy']}")
print("混淆矩阵:")
print(result['confusion_matrix'])
print("分类报告:")
print(result['classification_report'])
print("**********20内的最高价日*************")
columns_to_check = ['Pct_Chg_3d_later', 'Pct_Chg_5d_later', 'Pct_Chg_10d_later', 'Pct_Chg_20d_later']
df_lost['Max_Pct_Chg'] = df_lost[columns_to_check].max(axis=1)
df_lost['Target_max'] = (df_lost['Max_Pct_Chg'] > 0).astype(int)
result = logistic_regression_model(df_lost, ['Pct_Chg', 'Pct_Chg_szzs'], 'Target_max')
print(f"模型准确率: {result['accuracy']}")
print("混淆矩阵:")
print(result['confusion_matrix'])
print("分类报告:")
print(result['classification_report'])

***********3日*************
模型准确率: 0.5164609053497943
混淆矩阵:
[[495   5]
 [465   7]]
分类报告:
              precision    recall  f1-score   support

           0       0.52      0.99      0.68       500
           1       0.58      0.01      0.03       472

    accuracy                           0.52       972
   macro avg       0.55      0.50      0.35       972
weighted avg       0.55      0.52      0.36       972

***********5日*************
模型准确率: 0.5174897119341564
混淆矩阵:
[[502   3]
 [466   1]]
分类报告:
              precision    recall  f1-score   support

           0       0.52      0.99      0.68       505
           1       0.25      0.00      0.00       467

    accuracy                           0.52       972
   macro avg       0.38      0.50      0.34       972
weighted avg       0.39      0.52      0.36       972

***********10日*************
模型准确率: 0.5174897119341564
混淆矩阵:
[[480  24]
 [445  23]]
分类报告:
              precision    recall  f1-score   support

           0       0.52   

D:\anaconda3\envs\strategy\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
D:\anaconda3\envs\strategy\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
D:\anaconda3\envs\strategy\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

所有时间段的结果显示，模型严重偏向于预测负类，导致正类的识别能力极差。负类的召回率为100%并不代表模型性能好，因为正类被完全忽略。

LSTM算法用于股价预测                                          

支持向量机用于区分各特征值对于股价涨（1）和跌（0）的预测